<a href="https://colab.research.google.com/github/pginjupalli/Verizon-EnergyPriceForecasting/blob/main/Verizon_2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1: Understanding the Data**

Here, we will look at the dataset and note the features, their correlations with the label, whether or not there's missing data, and come up with ideas clearn and prepare the data for modeling.

In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

with open('/content/drive/My Drive/BTT Verizon 2/electricity_prices.csv', 'r') as f:
  df = pd.read_csv(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,year,month,stateDescription,sectorName,customers,price,revenue,sales
0,2001,1,Wyoming,all sectors,NaN,4.31,48.12840,1116.17208
1,2001,1,Wyoming,commercial,NaN,5.13,12.67978,247.08691
2,2001,1,Wyoming,industrial,NaN,3.26,19.60858,602.30484
3,2001,1,Wyoming,other,NaN,4.75,0.76868,16.17442
4,2001,1,Wyoming,residential,NaN,6.01,15.07136,250.60591
5,2001,1,New England,all sectors,NaN,10.73,1139.89520,10622.88375
6,2001,1,South Carolina,other,NaN,6.57,5.28332,80.41336
7,2001,1,South Carolina,residential,NaN,7.36,225.81673,3066.25911
8,2001,1,South Dakota,all sectors,NaN,5.85,46.98518,803.24177
9,2001,1,South Dakota,commercial,NaN,5.95,15.92044,267.66283


In [ ]:
df.shape

(85870, 8)

In [ ]:
df.head(50)

,year,month,stateDescription,sectorName,customers,price,revenue,sales
0,2001,1,Wyoming,all sectors,NaN,4.31,48.12840,1116.17208
1,2001,1,Wyoming,commercial,NaN,5.13,12.67978,247.08691
2,2001,1,Wyoming,industrial,NaN,3.26,19.60858,602.30484
3,2001,1,Wyoming,other,NaN,4.75,0.76868,16.17442
4,2001,1,Wyoming,residential,NaN,6.01,15.07136,250.60591
5,2001,1,New England,all sectors,NaN,10.73,1139.89520,10622.88375
6,2001,1,South Carolina,other,NaN,6.57,5.28332,80.41336
7,2001,1,South Carolina,residential,NaN,7.36,225.81673,3066.25911
8,2001,1,South Dakota,all sectors,NaN,5.85,46.98518,803.24177
9,2001,1,South Dakota,commercial,NaN,5.95,15.92044,267.66283


In [ ]:
print(df.isnull().sum()) # columns with null values
df[df['customers'].notnull()] # rows where 'customer' is not null

year                    0
month                   0
stateDescription        0
sectorName              0
customers           26040
price                   0
revenue                 0
sales                   0
dtype: int64


,year,month,stateDescription,sectorName,customers,price,revenue,sales
26040,2008,1,Washington,all sectors,3145488.0,6.64,574.73235,8658.35972
26041,2008,1,Rhode Island,transportation,0.0,0.00,0.00000,0.00000
26042,2008,1,South Carolina,transportation,0.0,0.00,0.00000,0.00000
26043,2008,1,Massachusetts,industrial,14142.0,13.18,100.40468,761.89017
26044,2008,1,Massachusetts,residential,2611383.0,16.81,315.43624,1876.02831
...,...,...,...,...,...,...,...,...
85865,2024,1,Arkansas,all sectors,1717720.0,9.63,442.98773,4598.63147
85866,2024,1,Arkansas,commercial,208669.0,10.26,97.79467,953.02154
85867,2024,1,Arkansas,industrial,34951.0,7.08,109.92656,1553.02838
85868,2024,1,Arkansas,residential,1474098.0,11.24,235.26399,2092.56172
